In [5]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
shivendratomar_5_sec_different_audio_path = kagglehub.dataset_download('shivendratomar/5-sec-different-audio')


print('Data source import complete.')


100%|██████████| 1.27G/1.27G [00:18<00:00, 74.4MB/s]

Extracting files...


Data source import complete.


In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
od.download('https://www.kaggle.com/datasets/shivendratomar/5-sec-different-audio')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shivendratomar
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/shivendratomar/5-sec-different-audio


100%|██████████| 1.27G/1.27G [00:06<00:00, 221MB/s]


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pickle

# For Kaggle input directory
file_path = '/content/5-sec-different-audio/prepared_data.pkl'

# Correct loading syntax
with open(file_path, 'rb') as f:
    prepared_data = pickle.load(f)  # Remove the parentheses after load

/tmp/ipython-input-4-2857649815.py:8: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  prepared_data = pickle.load(f)  # Remove the parentheses after load


In [5]:
def set_parameters():

    return {
        'data_dir': 'D:\CA content\Python\AISOC\Audio_Processing\EDA\DATASET',
        'sample_rate':22050,
        'duration': 5,  # seconds
        'n_mfcc': 40,
        'n_mels': 128,
        'n_fft': 2048,
        'hop_length': 512,
        'batch_size': 32,
        'epochs': 20,
        'validation_split': 0.2,
        'random_state': 42,
        'num_test_samples': 14  # Number of random samples to test
    }

In [6]:
params = set_parameters()

In [4]:
from tensorflow.keras.applications import ConvNeXtTiny
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Resizing, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [8]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Resizing, Conv2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf

def build_alternative_model(input_shape, num_categories, num_subcategories):
    # Load EfficientNetB0 as base model
    base_model = EfficientNetB0(
        include_top=False,
        input_shape=(224, 224, 3),
        weights="imagenet"
    )
    base_model.trainable = False

    # Input layer
    input_layer = Input(shape=input_shape)

    # Preprocessing
    x = Resizing(224, 224)(input_layer)
    x = Conv2D(3, (3, 3), padding="same", activation="relu")(x)  # Convert to 3 channels if input isn't RGB

    # Base model
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)

    # Shared features
    shared_features = Dense(256, activation="relu")(x)

    # Category branch
    category_output = Dense(128, activation="relu")(shared_features)
    category_output = Dropout(0.5)(category_output)
    category_output = Dense(num_categories, activation="softmax", name="category_output")(category_output)

    # Subcategory branch
    subcategory_output = Dense(128, activation="relu")(shared_features)
    subcategory_output = Dropout(0.5)(subcategory_output)
    subcategory_output = Dense(num_subcategories, activation="softmax", name="subcategory_output")(subcategory_output)

    # Build model
    model = Model(inputs=input_layer, outputs=[category_output, subcategory_output])

    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss={
            'category_output': 'categorical_crossentropy',
            'subcategory_output': 'categorical_crossentropy'
        },
        metrics={
            'category_output': 'accuracy',
            'subcategory_output': 'accuracy'
        }
    )

    return model

In [9]:
def train_model(prepared_data, params):
    num_categories = prepared_data['y_train_cat'].shape[1]
    num_subcategories = prepared_data['y_train_subcat'].shape[1]
    input_shape = prepared_data['X_train_mfcc'].shape[1:]

    model = build_alternative_model(input_shape, num_categories, num_subcategories)
    print(model.summary())

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=7,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=4,
            min_lr=0.000005
        )
    ]

    history = model.fit(
        prepared_data['X_train_mfcc'],
        {
            'category_output': prepared_data['y_train_cat'],
            'subcategory_output': prepared_data['y_train_subcat']
        },
        validation_data=(
            prepared_data['X_test_mfcc'],
            {
                'category_output': prepared_data['y_test_cat'],
                'subcategory_output': prepared_data['y_test_subcat']
            }
        ),
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        callbacks=callbacks
    )

    return model, history

In [10]:
model, history = train_model(prepared_data, params)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 40, 216,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resizing (Resizing) │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │         30 │ resizing[0][0]    │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 7, 7,      │  4,049,571 │ conv2d[0][0]      │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficientnetb0[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    327,936 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │     32,896 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_output     │ (None, 4)         │        516 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subcategory_output  │ (None, 14)        │      1,806 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,445,651 (16.96 MB)

 Trainable params: 396,080 (1.51 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

None
Epoch 1/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 91s 202ms/step - category_output_accuracy: 0.3718 - category_output_loss: 1.2503 - loss: 3.7052 - subcategory_output_accuracy: 0.1761 - subcategory_output_loss: 2.4548 - val_category_output_accuracy: 0.6934 - val_category_output_loss: 0.6969 - val_loss: 2.3183 - val_subcategory_output_accuracy: 0.4672 - val_subcategory_output_loss: 1.6217 - learning_rate: 0.0010
Epoch 2/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 24s 79ms/step - category_output_accuracy: 0.6919 - category_output_loss: 0.7270 - loss: 2.3393 - subcategory_output_accuracy: 0.4553 - subcategory_output_loss: 1.6123 - val_category_output_accuracy: 0.7821 - val_category_output_loss: 0.5439 - val_loss: 1.7285 - val_subcategory_output_accuracy: 0.5728 - val_subcategory_output_loss: 1.1842 - learning_rate: 0.0010
Epoch 3/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - category_output_accuracy: 0.7467 - category_output_loss: 0.5871 - loss: 1.8749 - subcategory_output_accuracy: 0.5547 - subcategory

In [16]:
def save_model(model, prepared_data, params, save_dir='saved_model'):

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    model.save(os.path.join(save_dir, 'audio_classifier.h5'))

    metadata = {
        'params': params,
        'category_mapping': prepared_data['category_mapping'],
        'subcategory_mapping': prepared_data['subcategory_mapping'],
        'category_to_idx': prepared_data['category_to_idx'],
        'subcategory_to_idx': prepared_data['subcategory_to_idx'],
        'input_shape': prepared_data['X_train_mfcc'].shape[1:],
    }

    np.save(os.path.join(save_dir, 'metadata.npy'), metadata)
    print(f"Model and metadata saved to {save_dir}")

In [17]:
import os
import numpy as np

def save_model(model, prepared_data, params, save_dir='saved_model'):
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)  # Better than manual check

    # Save model (HDF5 format)
    model_path = os.path.join(save_dir, 'audio_classifier.h5')
    model.save(model_path)

    # Prepare metadata
    metadata = {
        'params': params,
        'category_mapping': prepared_data['category_mapping'],
        'subcategory_mapping': prepared_data['subcategory_mapping'],
        'category_to_idx': prepared_data['category_to_idx'],
        'subcategory_to_idx': prepared_data['subcategory_to_idx'],
        'input_shape': prepared_data['X_train_mfcc'].shape[1:],
    }

    # Save metadata
    metadata_path = os.path.join(save_dir, 'metadata.npy')
    np.save(metadata_path, metadata)

    print(f"Model and metadata saved to {save_dir}")  # Fixed f-string

In [21]:
import os
save_dir = 'saved_model' # Define save_dir
model.save(os.path.join(save_dir, 'tf_savedmodel.keras'))

In [12]:
import os

In [23]:
# prompt: i wish to download metadata and save the model in h5 format.import os
# import pickle  # For saving metadata
# import numpy as np
# def save_model(model, prepared_data, params, save_dir='saved_model2'):
#     """
#     Saves the model (HDF5) and metadata (Pickle) to the specified directory.
#     Args:
#         model: Trained Keras model
#         prepared_data: Dictionary containing mappings and training data info
#         params: Model parameters/hyperparameters
#         save_dir: Directory to save files (default: 'saved_model')
#     """
#     # Create directory if it doesn't exist
#     os.makedirs(save_dir, exist_ok=True)
#     # --- Save Model (HDF5 format) ---
#     model_path = os.path.join(save_dir, 'audio_classifier.h5')
#     model.save(model_path)  # Saves as .h5 (Keras format)
#     # --- Save Metadata (Pickle format) ---
#     metadata = {
#         'params': params,
#         'category_mapping': prepared_data['category_mapping'],
#         'subcategory_mapping': prepared_data['subcategory_mapping'],
#         'category_to_idx': prepared_data['category_to_idx'],
#         'subcategory_to_idx': prepared_data['subcategory_to_idx'],
#         'input_shape': prepared_data['X_train_mfcc'].shape[1:],
#     }
#     metadata_path = os.path.join(save_dir, 'metadata.pkl')  # .pkl instead of .npy
#     with open(metadata_path, 'wb') as f:
#         pickle.dump(metadata, f)  # Save as pickle
#     print(f"Model (.h5) and metadata (.pkl) saved to: {save_dir}")

# The function `save_model` is already defined and handles saving in .h5 and .pkl formats.
# The task is to call this function after the model is trained.
# The model is trained by calling `model, history = train_model(prepared_data, params)`.
# We just need to call `save_model` with the trained model and the necessary data.

# The variable `save_dir` is already defined before the last `model.save` call.
# We should use this variable.

save_model(model, prepared_data, params, save_dir=save_dir)

Model (.h5) and metadata (.pkl) saved to: saved_model


In [7]:
!pip install tensorflow

In [25]:
import os
import pickle  # For saving metadata
import numpy as np

def save_model(model, prepared_data, params, save_dir='saved_model2'):
    """
    Saves the model (HDF5) and metadata (Pickle) to the specified directory.

    Args:
        model: Trained Keras model
        prepared_data: Dictionary containing mappings and training data info
        params: Model parameters/hyperparameters
        save_dir: Directory to save files (default: 'saved_model')
    """
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # --- Save Model (HDF5 format) ---
    model_path = os.path.join(save_dir, 'audio_classifier.h5')
    model.save(model_path)  # Saves as .h5 (Keras format)

    # --- Save Metadata (Pickle format) ---
    metadata = {
        'params': params,
        'category_mapping': prepared_data['category_mapping'],
        'subcategory_mapping': prepared_data['subcategory_mapping'],
        'category_to_idx': prepared_data['category_to_idx'],
        'subcategory_to_idx': prepared_data['subcategory_to_idx'],
        'input_shape': prepared_data['X_train_mfcc'].shape[1:],
    }

    metadata_path = os.path.join(save_dir, 'metadata.pkl')  # .pkl instead of .npy
    with open(metadata_path, 'wb') as f:
        pickle.dump(metadata, f)  # Save as pickle

    print(f"Model (.h5) and metadata (.pkl) saved to: {save_dir}")

In [24]:
# prompt: convert this h5 file to tensorflow.js

!pip install tensorflowjs

!tensorflowjs_converter \
    --input_format keras \
    '/content/saved_model/audio_classifier.h5' \
    '/content/tfjs_model'

print("Model converted to TensorFlow.js format in '/content/tfjs_model'")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.34.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


2025-07-17 10:57:39.020370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752749859.039883    8106 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752749859.046229    8106 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file
weight normalization/count with shape () and dtype int64 was auto converted to the type int32
Model converted to TensorFlow.js format in '/content/tfjs_model'


In [26]:
# prompt: download a folder tfjs_model

from google.colab import files
import shutil

# Define the directory to be downloaded
folder_to_download = 'tfjs_model'

# Check if the directory exists before trying to zip
if os.path.exists(folder_to_download):
    # Create a zip archive of the folder
    shutil.make_archive(folder_to_download, 'zip', folder_to_download)

    # Download the zip file
    files.download(f'{folder_to_download}.zip')
    print(f"Downloaded '{folder_to_download}.zip'")
else:
    print(f"Directory '{folder_to_download}' does not exist. Please ensure it was created successfully.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded 'tfjs_model.zip'


In [28]:
!tensorflowjs_converter --input_format=keras \
                       --output_format=tfjs_layers_model \
                       --quantize_float16 \
                       ./saved_model/tf_savedmodel.keras \
                       ./tfjs_model/

2025-07-17 11:05:59.942693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752750359.964221   10211 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752750359.970598   10211 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
Traceback (most recent call last):
  File "/usr/local/bin/tensorflowjs_converter", line 8, in <module>
    sys.exit(pip_main())
             ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tensorflowjs/converters/converter.py", line 959, in pip_main
    main([' '.join(sys.argv[1:])])
  File "/usr/local/lib